In [29]:
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, f1_score 
import sys
import pickle
import os
import time
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn

In [30]:
FOLDER_TOOLS = "/home/siamai/data/chuniji/ajanPLUSPLUS"
sys.path.append(FOLDER_TOOLS)
import Utils.feeder_skeleton as feeder_skeleton
from Utils.graph_model import Model
from Utils.Loss import ComboLoss

In [31]:
batch_size = 16 
test_data_path = "/home/siamai/data/Penguin/week7/ajart/outputnpy" # just for example
data_test_feeder = feeder_skeleton.Feeder(test_data_path)  # no label_path!
val_dataloader = DataLoader(data_test_feeder, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
def inference(model, dataloader, device):
    model.eval()
    results = []
    with torch.no_grad():
        for X, names in tqdm(dataloader):
            X = X.to(device)
            ypredict = model(X)  # [B, num_class]
            ypredict_sigmoid = torch.sigmoid(ypredict)
            ypredictlabel = (ypredict_sigmoid > 0.5).int().cpu().numpy()
            ypredict_prob = ypredict_sigmoid.cpu().numpy()
            for name, ypred, yprob in zip(names, ypredictlabel, ypredict_prob):
                results.append({"dataname": name, "prediction": ypred, "prob": yprob})
    return results
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Model creation
in_channels = 2
label_position = [0,1,2,3,4,5,6,7,8,9,10,11]
experiment_name = f"experiment_{'_'.join(map(str, label_position))}"
num_class = len(label_position) #adjust for 2 head 
graph_args = {
    "layout": "yolo",  
    "strategy": "uniform", 
    "max_hop": 1,  
    "dilation": 1  
}

edge_importance_weighting = False
kwargs = {}
threshold= 0.65

edge_importance_weighting = False
kwargs = {}
model = Model(in_channels, num_class, graph_args, edge_importance_weighting, **kwargs)
model = model.to(device)

weight_path = "/home/siamai/data/Penguin/week7/ajart/weight/experiment_0_1_2_3_4_5_6_7_8_9_10_1145_F1macro0.3418.pth"
model.load_state_dict(torch.load(weight_path, map_location=device))
print(f"Loaded weights from {weight_path}")

# Inference
results = inference(model, val_dataloader, device)
# id,Filename,Time,Advancing,Retreating,Enclosing,Spreading,Rising,Descending,Directing,Indirecting,Increasing Pressure,Decreasing Pressure,Acceleration,Decelerating
# Example: Save as CSV
cls_list = ['Advancing','Retreating','Enclosing','Spreading','Rising','Descending','Directing','Indirecting','Increasing Pressure','Decreasing Pressure','Acceleration','Decelerating']
import pandas as pd
df = pd.DataFrame([
    {
        "Filename": r["dataname"].replace('.npy', '').rsplit('_', 1)[0],
        "Time": r["dataname"].replace('.npy', '').rsplit('_', 1)[1],
        **{f"{cls_list[i]}": r["prediction"][i] for i in range(len(r["prediction"]))}
    }
    for r in results
])
df.to_csv(f"/home/siamai/data/Penguin/week7/inference/result/predict_{experiment_name}.csv", index=False)

/tmp/ipykernel_477738/2235684167.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_path, map_location=device))


A: torch.Size([1, 17, 17])
Loaded weights from /home/siamai/data/Penguin/week7/ajart/weight/experiment_0_1_2_3_4_5_6_7_8_9_10_1145_F1macro0.3418.pth


  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:00<00:00, 50.62it/s]


In [14]:
df

,Filename,Time,Advancing,Retreating,Enclosing,Spreading,Rising,Descending,Directing,Indirecting,Increasing Pressure,Decreasing Pressure,Acceleration,Decelerating
0,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1621,1,0,0,0,0,0,1,0,0,1,0,1
1,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1647,1,0,1,0,0,0,1,0,0,1,0,1
2,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1650,1,0,0,0,0,0,1,0,0,1,0,1
3,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1706,1,0,0,0,0,0,1,0,0,1,0,1
4,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1709,1,0,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,s06_smiley,t3000,1,0,0,1,0,0,1,0,1,0,1,0
276,s06_smiley,t3014,1,0,0,1,0,0,1,0,1,0,1,0
277,s06_smiley,t3019,1,0,0,1,0,0,1,0,1,0,1,0
278,s06_smiley,t3034,1,0,0,1,0,0,1,0,1,0,1,0


In [7]:
df.loc[0,'dataname']

'Tom_Cruise_on_Mission_Impossible_and_a_lifetime_of_learning_from_movies_BFI_in_Conversation_t1621.npy'

In [9]:
filename = 'Tom_Cruise_on_Mission_Impossible_and_a_lifetime_of_learning_from_movies_BFI_in_Conversation_t1621.npy'

# Remove .npy extension
basename = filename.replace('.npy', '')

# Split by the last underscore
main_part, time_part = basename.rsplit('_', 1)

print("Main part:", main_part)
print("Time part:", time_part)


Main part: Tom_Cruise_on_Mission_Impossible_and_a_lifetime_of_learning_from_movies_BFI_in_Conversation
Time part: t1621


In [55]:
test = pd.read_csv('/home/siamai/data/week7/test_submission.csv')
test

,id,Filename,Time,Advancing,Retreating,Enclosing,Spreading,Rising,Descending,Directing,Indirecting,Increasing Pressure,Decreasing Pressure,Acceleration,Decelerating
0,cdd7e8f4-825f-4e6a-a0ba-a12a4ecf8676,s06_smiley,t0440,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,aebb87da-7d6e-487d-be99-e93c8502f72a,s06_smiley,t0442,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,42aea8ef-38d9-4bfe-9dc3-2a890d43ab16,s06_smiley,t0459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,68abb24e-54e3-40a9-ab6d-ca4d5ddff1f5,s06_smiley,t0528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d2ee103d-4fd8-4440-91ca-41b08ff1be51,s06_smiley,t0530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,a1340763-f06c-4e70-8051-3d88fa8b0100,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t3007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276,5d45adfb-7ec1-4da7-9a72-fadf120d73f4,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t3008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277,90ab423d-ff44-4ca6-9a2c-77a4fc01e1ee,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t3009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,f22cec6b-70ea-4d9d-bf62-fd596243bcd0,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t3016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['id'] = test['id']

In [16]:
df.to_csv(f"/home/siamai/data/Penguin/week7/inference/result/predict_{experiment_name}_edited.csv", index=False)

In [17]:
import pandas as pd

# Read your CSVs or dataframes
# df1 = pd.read_csv('file1.csv')
# df2 = pd.read_csv('file2.csv')

# Example dataframes for demo:
# df1 = pd.DataFrame({'Filename': ['a', 'b'], 'Time': [1, 2]})
# df2 = pd.DataFrame({'id': [100, 200], 'Filename': ['a', 'b'], 'Time': [1, 2]})

# Merge on both columns
df_merged = df.merge(test[['id', 'Filename', 'Time']], on=['Filename', 'Time'], how='left')

# Now df_merged has the 'id' column added
df_merged

,Filename,Time,Advancing,Retreating,Enclosing,Spreading,Rising,Descending,Directing,Indirecting,Increasing Pressure,Decreasing Pressure,Acceleration,Decelerating,id
0,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1621,1,0,0,0,0,0,1,0,0,1,0,1,b6fc0dc8-e807-4c15-b9ac-74e335c27785
1,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1647,1,0,1,0,0,0,1,0,0,1,0,1,4ff9813a-77ef-4885-aba9-9d36ff3a502a
2,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1650,1,0,0,0,0,0,1,0,0,1,0,1,096aaef7-703f-4ca0-90eb-14c1b3a4114b
3,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1706,1,0,0,0,0,0,1,0,0,1,0,1,22252282-e185-4590-b150-2e74dc81dfc6
4,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1709,1,0,0,0,0,0,1,0,0,1,0,1,4e2d9d78-6eca-4f1f-98aa-139166f70c6e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,s06_smiley,t3000,1,0,0,1,0,0,1,0,1,0,1,0,f00a9fd8-174f-4777-ad7b-e17998240151
276,s06_smiley,t3014,1,0,0,1,0,0,1,0,1,0,1,0,b266a8fc-4a1d-457a-80cc-58ec565c15ba
277,s06_smiley,t3019,1,0,0,1,0,0,1,0,1,0,1,0,4ec01160-df2d-4fbd-af44-12f3db4a0948
278,s06_smiley,t3034,1,0,0,1,0,0,1,0,1,0,1,0,b5a3fe57-b7fe-4c2d-96b1-e1f0c51cebb4


In [19]:
df_merged.to_csv(f"/home/siamai/data/Penguin/week7/inference/result/predict_{experiment_name}_edited1.csv", index=False)

In [87]:
def inference(model, dataloader, device):
    model.eval()
    results = []
    with torch.no_grad():
        for X, names in tqdm(dataloader):
            X = X.to(device)
            ypredict = model(X)  # [B, num_class]
            ypredict_sigmoid = torch.sigmoid(ypredict)
            ypredictlabel = (ypredict_sigmoid > 0.5).int().cpu().numpy()
            ypredict_prob = ypredict_sigmoid.cpu().numpy()
            for name, ypred, yprob in zip(names, ypredictlabel, ypredict_prob):
                results.append({"dataname": name, "prediction": ypred, "prob": yprob})
    return results
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Model creation
in_channels = 2
label_position = [10,11]#[0,1,2,3,4,5,6,7,8,9,10,11]
experiment_name = f"experiment_{'_'.join(map(str, label_position))}"
num_class = len(label_position) #adjust for 2 head 
graph_args = {
    "layout": "yolo",  
    "strategy": "uniform", 
    "max_hop": 1,  
    "dilation": 1  
}
weight_path = "/home/siamai/data/Penguin/week7/ajart/weight/experiment_10_11_21_F1macro0.4970.pth"
all_cls_list = ['Advancing','Retreating','Enclosing','Spreading','Rising','Descending','Directing','Indirecting','Increasing Pressure','Decreasing Pressure','Acceleration','Decelerating']
edge_importance_weighting = True

def predict(in_channels, label_position, weight_path):
    num_class = len(label_position)
    kwargs = {}
    threshold= 0.65

    
    kwargs = {}
    model = Model(in_channels, num_class, graph_args, edge_importance_weighting, **kwargs)
    model = model.to(device)

    model.load_state_dict(torch.load(weight_path, map_location=device))
    print(f"Loaded weights from {weight_path}")
    cls_list = [all_cls_list[i] for i in label_position]

    # Inference
    results = inference(model, val_dataloader, device)
    # id,Filename,Time,Advancing,Retreating,Enclosing,Spreading,Rising,Descending,Directing,Indirecting,Increasing Pressure,Decreasing Pressure,Acceleration,Decelerating
    # Example: Save as CSV
    import pandas as pd
    df = pd.DataFrame([
        {
            "Filename": r["dataname"].replace('.npy', '').rsplit('_', 1)[0],
            "Time": r["dataname"].replace('.npy', '').rsplit('_', 1)[1],
            **{f"{cls_list[i]}": r["prediction"][i] for i in range(len(r["prediction"]))}
        }
        for r in results
    ])
    df.to_csv(f"/home/siamai/data/Penguin/week7/inference/result/predict_add{experiment_name}.csv", index=False)
    return df

In [88]:
df = predict(in_channels, label_position, weight_path)

A: torch.Size([1, 17, 17])


/tmp/ipykernel_477738/3319789216.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weight_path, map_location=device))


Loaded weights from /home/siamai/data/Penguin/week7/ajart/weight/experiment_10_11_21_F1macro0.4970.pth


  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:00<00:00, 52.13it/s]


In [86]:
df

,Filename,Time,Increasing Pressure,Decreasing Pressure
0,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1621,0,1
1,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1647,0,1
2,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1650,0,1
3,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1706,0,1
4,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1709,0,1
...,...,...,...,...
275,s06_smiley,t3000,1,0
276,s06_smiley,t3014,1,0
277,s06_smiley,t3019,1,0
278,s06_smiley,t3034,1,0


In [89]:
df1 = pd.read_csv('/home/siamai/data/Penguin/week7/inference/result/predict_addexperiment_0_1.csv')

In [90]:
df2 = pd.read_csv('/home/siamai/data/Penguin/week7/inference/result/predict_addexperiment_2_3.csv')

In [91]:
df3 = pd.read_csv('/home/siamai/data/Penguin/week7/inference/result/predict_addexperiment_4_5.csv')

In [92]:
df4 = pd.read_csv('/home/siamai/data/Penguin/week7/inference/result/predict_addexperiment_6_7.csv')

In [93]:
df5= pd.read_csv('/home/siamai/data/Penguin/week7/inference/result/predict_addexperiment_8_9.csv')

In [94]:
df6= pd.read_csv('/home/siamai/data/Penguin/week7/inference/result/predict_addexperiment_10_11.csv')

In [ ]:
all_cls_list = ['Advancing','Retreating','Enclosing','Spreading','Rising','Descending','Directing','Indirecting','Increasing Pressure','Decreasing Pressure','Acceleration','Decelerating']


In [95]:
df1['Enclosing'] = df2['Enclosing']
df1['Spreading'] = df2['Spreading']
df1['Rising'] = df3['Rising']
df1['Descending'] = df3['Descending']
df1['Directing'] = df4['Directing']
df1['Indirecting'] = df4['Indirecting']
df1['Increasing Pressureg'] = df5['Increasing Pressure']
df1['Decreasing Pressure'] = df5['Decreasing Pressure']
df1['Acceleration'] = df6['Acceleration']
df1['Decelerating'] = df6['Decelerating']

In [96]:
df1

,Filename,Time,Advancing,Retreating,Enclosing,Spreading,Rising,Descending,Directing,Indirecting,Increasing Pressureg,Decreasing Pressure,Acceleration,Decelerating
0,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1621,0,0,0,1,0,0,1,0,0,1,0,0
1,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1647,0,0,1,0,0,0,1,0,0,1,0,0
2,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1650,0,0,1,0,0,0,1,0,0,1,0,0
3,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1706,1,0,0,1,0,0,1,0,0,1,0,0
4,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1709,1,0,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,s06_smiley,t3000,0,0,1,0,0,0,0,0,1,0,1,0
276,s06_smiley,t3014,0,0,1,0,0,0,0,0,1,0,1,0
277,s06_smiley,t3019,0,0,1,0,0,0,0,0,1,0,1,0
278,s06_smiley,t3034,0,0,1,0,0,0,0,0,1,0,1,0


In [97]:
sub = df1.merge(test[['id', 'Filename', 'Time']], on=['Filename', 'Time'], how='left')
sub

,Filename,Time,Advancing,Retreating,Enclosing,Spreading,Rising,Descending,Directing,Indirecting,Increasing Pressureg,Decreasing Pressure,Acceleration,Decelerating,id
0,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1621,0,0,0,1,0,0,1,0,0,1,0,0,b6fc0dc8-e807-4c15-b9ac-74e335c27785
1,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1647,0,0,1,0,0,0,1,0,0,1,0,0,4ff9813a-77ef-4885-aba9-9d36ff3a502a
2,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1650,0,0,1,0,0,0,1,0,0,1,0,0,096aaef7-703f-4ca0-90eb-14c1b3a4114b
3,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1706,1,0,0,1,0,0,1,0,0,1,0,0,22252282-e185-4590-b150-2e74dc81dfc6
4,Tom_Cruise_on_Mission_Impossible_and_a_lifetim...,t1709,1,0,0,1,0,0,1,0,0,1,0,0,4e2d9d78-6eca-4f1f-98aa-139166f70c6e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,s06_smiley,t3000,0,0,1,0,0,0,0,0,1,0,1,0,f00a9fd8-174f-4777-ad7b-e17998240151
276,s06_smiley,t3014,0,0,1,0,0,0,0,0,1,0,1,0,b266a8fc-4a1d-457a-80cc-58ec565c15ba
277,s06_smiley,t3019,0,0,1,0,0,0,0,0,1,0,1,0,4ec01160-df2d-4fbd-af44-12f3db4a0948
278,s06_smiley,t3034,0,0,1,0,0,0,0,0,1,0,1,0,b5a3fe57-b7fe-4c2d-96b1-e1f0c51cebb4


In [98]:
import pandas as pd

# โหลดไฟล์ inference ผลลัพธ์ CSV
df = sub

# กำหนดคู่ตรงข้ามเป็น list ของ tuple (ชื่อคอลัมน์)
opposite_pairs = [
    ("Advancing", "Retreating"),
    ("Spreading", "Enclosing"),
    ("Rising", "Descending"),
    ("Acceleration", "Decelerating"),
    ("Directing", "Indirecting"),
    ("Increasing Pressureg", "Decreasing Pressure")
]

total_fixed = 0

# ทำ post-process
for col1, col2 in opposite_pairs:
    # หาแถวที่ทั้งคู่เป็น 1
    both_one = (df[col1] == 1) & (df[col2] == 1)
    count_fix = both_one.sum()
    if count_fix > 0:
        # แก้ไขเป็น 0,0
        df.loc[both_one, [col1, col2]] = 0
        print(f"Fixed {count_fix} rows for pair ({col1}, {col2})")
        total_fixed += count_fix

print(f"Total rows fixed across all pairs: {total_fixed}")

# บันทึกไฟล์ใหม่
df.to_csv("test_predictions_postprocessed.csv", index=False)

print("Post-processing complete and saved to test_predictions_postprocessed.csv")


Fixed 1 rows for pair (Spreading, Enclosing)
Total rows fixed across all pairs: 1
Post-processing complete and saved to test_predictions_postprocessed.csv
